In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import accuracy_score
import anndata as ad
import pandas as pd
import anndata as ad
import pandas as pd
import numpy as np

import umap
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import calinski_harabasz_score

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
input_dir = "/Users/apple/Desktop/KB/data"
adata_train = ad.read_h5ad(input_dir+'/LarryData/train_test/Larry_500_train_lineage.h5ad')
adata_test = ad.read_h5ad(input_dir+'/LarryData/train_test/Larry_500_test_lineage.h5ad')


train_labels = adata_train.obs["clone_id"].to_numpy()
test_labels = adata_test.obs["clone_id"].to_numpy()
print(train_labels.shape, test_labels.shape)

(17422,) (1809,)


## supUMAP embedding

In [3]:
# Extract the data matrix and labels
train_data = adata_train.X
test_data = adata_test.X


# labels = adata_train.obs['clone_id'].values

# Initialize UMAP with a higher number of neighbors for supervised learning
reducer = umap.UMAP(n_neighbors=15, n_components=10)



In [4]:
# Fit and transform the data with the labels
train_embeddings = reducer.fit_transform(train_data, y=train_labels)

In [5]:
# get the test embeddings
test_embeddings = reducer.transform(test_data)

In [6]:
test_embeddings.shape, train_embeddings.shape

((1809, 10), (17422, 10))

In [7]:
# Save the embeddings to a .npy file
np.save("supUMAP_biddy_test_embedding.npy", test_embeddings)
np.save("supUMAP_biddy_train_embedding.npy", train_embeddings)
# Load the embeddings from the .npy file
# test_embeddings = np.load("supUMAP_biddy_test_embedding.npy")
# train_embeddings = np.load("supUMAP_biddy_train_embedding.npy")


#### calinski score

In [8]:
# Calculate the Calinski-Harabasz score
score = calinski_harabasz_score(train_embeddings, train_labels)

# Print the score
print("Train Calinski-Harabasz Score:", score)


Train Calinski-Harabasz Score: 94.15854848240309


In [9]:
# Calculate the Calinski-Harabasz score
score = calinski_harabasz_score(test_embeddings, test_labels)

# Print the score
print("Test Calinski-Harabasz Score:", score)


Test Calinski-Harabasz Score: 100.83666317408894


## KNN classifier

In [18]:
knn_graph = kneighbors_graph(test_embeddings, 5, mode='connectivity', include_self=True)
knn_graph.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [19]:
knn_graph.shape

(1809, 1809)

In [20]:
accuracy_vector = np.zeros(knn_graph.shape[0])

# Loop over each row (test sample)
for i in range(knn_graph.shape[0]):
    # Convert the sparse row to a dense array
    knn_row_dense = knn_graph[i].toarray().flatten()  # Flatten to ensure it's 1D
    
    # Get the indices of the neighbors for the i-th test sample
    neighbor_indices = np.where(knn_row_dense > 0)[0]
    
    # Get the labels of the neighbors
    neighbor_labels = test_labels[neighbor_indices]
    
    # Compare the neighbor labels to the label of the current test sample
    correct_matches = np.sum(neighbor_labels == test_labels[i])
    
    # Calculate the percentage of correct matches
    accuracy_vector[i] = correct_matches / len(neighbor_indices)




In [21]:
accuracy_vector.mean()

0.3587617468214484